<a href="https://colab.research.google.com/github/Singular-Brain/bindsnet/blob/master/lc_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook setups

In [1]:
!pip install -q git+https://github.com/Singular-Brain/bindsnet

     |████████████████████████████████| 120 kB 4.0 MB/s 
     |████████████████████████████████| 76 kB 3.3 MB/s 
     |████████████████████████████████| 72 kB 927 kB/s 
     |████████████████████████████████| 280 kB 6.7 MB/s 
     |████████████████████████████████| 28.5 MB 39 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
!wget https://data.deepai.org/mnist.zip
!mkdir -p ../data/MNIST/TorchvisionDatasetWrapper/raw
!unzip mnist.zip -d ../data/MNIST/TorchvisionDatasetWrapper/raw/

--2021-08-05 08:01:01--  https://data.deepai.org/mnist.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11597176 (11M) [application/x-zip-compressed]
Saving to: ‘mnist.zip’

mnist.zip           100%[===================>]  11.06M  4.58MB/s    in 2.4s    

2021-08-05 08:01:05 (4.58 MB/s) - ‘mnist.zip’ saved [11597176/11597176]

Archive:  mnist.zip
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-images-idx3-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-labels-idx1-ubyte.gz  


In [6]:
from bindsnet.network.nodes import Nodes
import os
import copy
import time
import torch
import random
import torchvision
import numpy as np
import argparse
import matplotlib.pyplot as plt
import collections
import manual_seed
from torchvision import transforms
from tqdm.notebook import tqdm

from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes, AdaptiveLIFNodes
from bindsnet.network.topology import LocalConnection, Connection
from bindsnet.network.monitors import Monitor, AbstractMonitor, TensorBoardMonitor
from bindsnet.learning import PostPre, MSTDP, MSTDPET 
from bindsnet.learning.reward import DynamicDopamineInjection
from bindsnet.utils import get_square_assignments, get_square_weights
from bindsnet.evaluation import all_activity, proportion_weighting, assign_labels

from bindsnet.analysis.plotting import (
    plot_input,
    plot_assignments,
    plot_performance,
    plot_weights,
    plot_spikes,
    plot_voltages,
)

## Sets up Gpu use


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu = True
train = True
def manual_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
            
torch.set_num_threads(os.cpu_count() - 1)
print("Running on Device = ", device)

if not train:
    update_interval = n_test
seed = 2045 # The Singularity is Near!
manual_seed(seed)

Running on Device =  cpu


## Set up hyper-parameters

<a href="https://imgflip.com/i/5in8rq"><img src="https://i.imgflip.com/5in8rq.jpg" title="made at imgflip.com"/></a><div><a href="https://imgflip.com/memegenerator"></a>By Mahbod</div>

In [8]:
seed = 2045 # The Singularity is Near!
manual_seed(seed)
# training hyperparameters
n_train = 200
n_test = 100
n_val = 50
val_interval = 50
running_window_length = 50
time = 250
dt = 1
train = True
gpu = False
device_id = 0

# Dataset Hyperparameters
target_classes = (0,1)
if target_classes:
    npz_file =np.load(f'mask_{target_classes[0]}_{target_classes[1]}.npz')
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10
crop_size = 20
intensity = 127.5 

# Network Architecture Hyperparameters 
n_neurons = 100
C = 50
K = 12
S = 4
theta_plus = 0.05   ## Adaptive LIF
inh_factor = 0.5
wmin = -1.0
wmax = 1.0
neuron_per_class = int(n_neurons/n_classes)

train_hparams = {
    'n_train' : n_train,
    'n_test' : n_test,
    'n_val' : n_val,
    'val_interval' : val_interval,
    'time' : time, 
    'dt' : dt, 
    'running_window_length': running_window_length,
}

data_hparams = { 
     'n_classes': n_classes,
    'intensity': intensity,
    'crop_size': crop_size,
}

network_hparams = {
    'n_neurons' : n_neurons,
    'n_channels': C,
    'filter_size': K,
    'stride': S,
    'theta_plus': theta_plus,
    'inh_factor': inh_factor,
    'wmin': wmin,
    'wmax': wmax,
    'n_neuron_per_class': neuron_per_class,
}

# reward Hyperparameters 
reward_hparams = {
    'dopaminergic_layer': 'output', 
    'n_labels': n_classes,
    'neuron_per_class': neuron_per_class,
    'dopamine_per_spike': 0.001,
    'dopamine_for_correct_pred': 0.0,
    'tc_reward': 20,
    'dopamine_base': 0.002,
    'variant': 'default'
}

# Design network

## Reward Monitor

In [9]:
class RewardMonitor(AbstractMonitor):
    # language=rst
    """
    Records state variables of interest.
    """

    def __init__(
        self,
        time: None,
        batch_size: int = 1,
        device: str = "cpu",
    ):
        # language=rst
        """
        Constructs a ``Monitor`` object.

        :param obj: An object to record state variables from during network simulation.
        :param state_vars: Iterable of strings indicating names of state variables to record.
        :param time: If not ``None``, pre-allocate memory for state variable recording.
        :param device: Allow the monitor to be on different device separate from Network device
        """
        super().__init__()

        self.time = time
        self.batch_size = batch_size
        self.device = device

        # if time is not specified the monitor variable accumulate the logs
        if self.time is None:
            self.device = "cpu"

        self.recording = []
        self.reset_state_variables()

    def get(self,) -> torch.Tensor:
        # language=rst
        """
        Return recording to user.

        :return: Tensor of shape ``[time, n_1, ..., n_k]``, where ``[n_1, ..., n_k]`` is the shape of the recorded state
        variable.
        Note, if time == `None`, get return the logs and empty the monitor variable

        """
        # return_logs = torch.as_tensor(self.recording)
        # if self.time is None:
        #     self.recording = []
        return self.recording

    def record(self, **kwargs) -> None:
        # language=rst
        """
        Appends the current value of the recorded state variables to the recording.
        """
        self.recording.append(kwargs["reward"])
        # remove the oldest element (first in the list)
        # if self.time is not None:
        #     self.recording.pop(0)

    def reset_state_variables(self) -> None:
        # language=rst
        """
        Resets recordings to empty ``List``s.
        """
        self.recording = []


In [10]:
manual_seed(seed)

compute_size = lambda inp_size, k, s: int((inp_size-k)/s) + 1

### Reward function
reward_fn = DynamicDopamineInjection
network = Network(dt = 1, reward_fn = reward_fn)

### nodes
inp = Input(shape= [1,20,20], traces=True)

main = LIFNodes(shape= [C, compute_size(crop_size, K, S), compute_size(crop_size, K, S)], traces=True)
#main = LIFNodes(shape= [C, compute_size(20, K, S), compute_size(20, K, S)], traces=True)

# TODO: Diehl & Cook 2015 (v2) 
out = LIFNodes(n= 100, traces=True, tc_trace=20)

### connections 
LC = LocalConnection(inp, main, K, S, C, nu = 1e-2, update_rule = MSTDPET, wmin = wmin, wmax= wmax, )
#LC.w = torch.nn.init.xavier_normal_(LC.w, gain=1.0)
main_out = Connection(main, out, nu = 1e-2, update_rule = MSTDPET, wmin = wmin, wmax= wmax, )
#main_out.w = torch.nn.init.xavier_normal_(main_out.w, gain=1.0)


w = -inh_factor * torch.ones(out.n, out.n)
for c in range(n_classes):
    ind = slice(c*neuron_per_class,(c+1)*neuron_per_class)
    w[ind, ind] = 0


out_recurrent_inhibition = Connection(
    source=out,
    target=out,
    w=w,
    wmin=-inh_factor,
    wmax=0,
    nu = 1e-4,
)

network.add_layer(main, "main")
network.add_layer(inp, "input")
network.add_layer(out, "output")
network.add_connection(LC, "input", "main")
network.add_connection(main_out, "main", "output")
network.add_connection(out_recurrent_inhibition, "output", "output")

# Directs network to GPU
if gpu:
    network.to("cuda")

# Voltage recording for excitatory and inhibitory layers.
main_monitor = Monitor(network.layers["main"], ["v"], time=time, device=device)
output_monitor = Monitor(network.layers["output"], ["v"], time=time, device=device)
reward_monitor = RewardMonitor(time =time)
tensorboard = TensorBoardMonitor(network, time = time)
network.add_monitor(main_monitor, name="main")
network.add_monitor(output_monitor, name="output")
network.add_monitor(reward_monitor, name="reward")
network.add_monitor(tensorboard, name="tensorboard")



# Load Dataset

In [11]:
class ClassSelector(torch.utils.data.sampler.Sampler):
    """Select target classes from the dataset"""
    def __init__(self, target_classes, data_source, mask = None):
        if mask is not None:
            self.mask = mask
        else:
            self.mask = torch.tensor([1 if data_source[i]['label'] in target_classes else 0 for i in range(len(data_source))])
        self.data_source = data_source

    def __iter__(self):
        return iter([i.item() for i in torch.nonzero(self.mask)])

    def __len__(self):
        return len(self.data_source)

In [14]:
# Load MNIST data.
manual_seed(seed)
dataset = MNIST(
    PoissonEncoder(time=time, dt=dt,),
    None,
    root=os.path.join("..", "..", "data", "MNIST"),
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(),
        transforms.Lambda(lambda x: x * intensity),
        transforms.CenterCrop(crop_size)]
    ),
)

# Create a dataloader to iterate and batch data
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                         sampler = ClassSelector(
                                                target_classes = target_classes,
                                                data_source = dataset,
                                                mask = mask,
                                                ) if target_classes else None
                                         )

# Load test dataset
test_dataset = MNIST(
    PoissonEncoder(time=time, dt=dt),
    None,
    root=os.path.join("..", "..", "data", "MNIST"),
    download=True,
    train=False,
    transform=transforms.Compose(
        [transforms.ToTensor(),
        transforms.Lambda(lambda x: x * intensity),
        transforms.CenterCrop(crop_size)]
    ),
)

val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                         sampler = ClassSelector(
                                                target_classes = target_classes,
                                                data_source = test_dataset,
                                                mask = mask_test,
                                                ) if target_classes else None
                                         )

HTTPError: HTTP Error 503: Service Unavailable

# Evaluation Utils

In [14]:
def evaluate(network, val_loader, spikes):
    acc_hist_val = collections.deque([], running_window_length)

    spikes_val = {}

    network.train = False
    for (i, datum) in enumerate(val_loader):
        if i > n_val:
            break

        image = datum["encoded_image"]
        label = datum["label"]

        # Run the network on the input.
        if gpu:
            inputs = {"input": image.cuda().view(time, 1, 1, 20, 20)}
        else:
            inputs = {"input": image.view(time, 1, 1, 20, 20)}
        network.run(inputs=inputs, time=time, **reward_hparams, labels=label, train=False)

        # Add to spikes recording.
        output_spikes = spikes["output"].get("s").view(time, n_classes, neuron_per_class).sum(0)
        predicted_label = torch.argmax(output_spikes.sum(1))

        if predicted_label == label:
            acc_hist_val.append(1)
        else:
            acc_hist_val.append(0)
        

        network.reset_state_variables()  # Reset state variables.

    network.train = True
    val_acc = 100 * sum(acc_hist_val)/len(acc_hist_val)
    return val_acc

# Train

In [15]:
def create_plot(spikes, reward, label):
    fig = plt.figure()
    ax = fig.gca()
    for i in range(spikes.shape[1]):
        spikes_sum = spikes.sum(-1)[:,i]
        spike_timepoints = np.where(spikes_sum)[0]
        spike_values = spikes_sum[spike_timepoints]
        if i == label:
            kwargs = {'s':10, 'marker' : '*', 'c' : 'r'}
        else:
            kwargs = {'s':5, 'marker': '*'}
        
        ax.scatter(spike_timepoints, spike_values, **kwargs)
    
    ax.plot(reward)
    return fig

In [16]:
# Train the network.
print("Begin training.\n")
acc_hist = collections.deque([], running_window_length)
manual_seed(seed)

spikes = {}
for layer in set(network.layers):
    spikes[layer] = Monitor(network.layers[layer], state_vars=["s"], time=time)
    network.add_monitor(spikes[layer], name="%s_spikes" % layer)

val_acc = 0.0

reward_history = []
pbar = tqdm(total=n_train)
network.reset_state_variables()
for (i, datum) in enumerate(dataloader):
    if i > n_train:
        break

    image = datum["encoded_image"]
    label = datum["label"]

    # Run the network on the input.
    if gpu:
        inputs = {"input": image.cuda().view(time, 1, 1, 20, 20)}
    else:
        inputs = {"input": image.view(time, 1, 1, 20, 20)}
    network.run(inputs=inputs, time=time, **reward_hparams, labels = label, train=True)


    # Weight Normalization


    # Get voltage recording.
    main_voltage = main_monitor.get("v")
    out_voltage = output_monitor.get("v")
    reward_history.append(reward_monitor.get())
    tensorboard.update(step= i)

    # Add to spikes recording.
    output_spikes = spikes["output"].get("s").view(time, n_classes, neuron_per_class)
    predicted_label = torch.argmax(output_spikes.sum(0).sum(1))

    print("\routput", output_spikes.sum(0).sum(1), 'predicted_label:',
          predicted_label.item(), 'GT:', label.item(), ' Reward:',
          sum(reward_monitor.get()).item(),
          end = '')

    if  i % val_interval == 0 and i!=0:
        val_acc = evaluate(network, val_loader, spikes)

    if  i % val_interval == 0 and i!=0:
        fig = create_plot(output_spikes, reward_monitor.get(), label)
        tensorboard.writer.add_figure('reward', fig, i)

    if predicted_label == label:
        acc_hist.append(1)
    else:
        acc_hist.append(0)
        
    acc = 100 * sum(acc_hist)/len(acc_hist)
    network.reset_state_variables()  # Reset state variables.
    
    pbar.set_description_str("Running accuracy: " + "{:.2f}".format(acc) + "%, " + "Current val accuracy: " + "{:.2f}".format(val_acc) + "%, ")
    pbar.update()

result_metrics = {'train_acc': acc, 'val_acc': val_acc}
tensorboard.writer.add_hparams(
    {**train_hparams, **data_hparams, **network_hparams, **reward_hparams},
    result_metrics
)

Begin training.



output tensor([24, 32]) predicted_label: 1 GT: 1  Reward: 1.0757639408111572

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs

# Kernel 

In [ ]:
from abc import ABC, abstractmethod
from typing import Union, Tuple, Optional, Sequence
from torch.nn.modules.utils import _pair

In [ ]:
class AbstractKernel(ABC):
	r"""Base class for generating image filter kernels such as Gabor, DoG, etc. Each subclass should override :attr:`__call__` function.
	"""
	def __init__(self, kernel_size: Union[int, Tuple[int, int]]):
    """
    Instantiates a ``Filter Kernel`` object.

    :param kernel_size: Horizontal and vertical size of convolutional kernels.
    """
		self.kernel_size = _pair(kenel_size)

	def __call__(self):
		pass


In [ ]:
class DoG_Kernel(AbstractKernel):
	r"""Generates DoG filter kernels.
	"""
	def __init__(self, 
                 kernel_size: Union[int, Tuple[int, int]], 
                 sigma1 : float,
                 sigma2 : float):
        """
		:param kernel_size: Horizontal and vertical size of DOG kernels.(If pass int, we consider it as a square filter) 
		:param sigma1 : The sigma parameter for the first Gaussian function.
		:param sigma2 : The sigma parameter for the second Gaussian function.
		"""
        super(DoG_Kernel, self).__init__(kernel_size)
		self.sigma1 = sigma1
		self.sigma2 = sigma2

	# returns a 2d tensor corresponding to the requested DoG filter
	def __call__(self):
		k = self.kernel_size//2
		x, y = np.mgrid[-k:k+1:1, -k:k+1:1]
		a = 1.0 / (2 * math.pi)
		prod = x*x + y*y
		f1 = (1/(self.sigma1*self.sigma1)) * np.exp(-0.5 * (1/(self.sigma1*self.sigma1)) * (prod))
		f2 = (1/(self.sigma2*self.sigma2)) * np.exp(-0.5 * (1/(self.sigma2*self.sigma2)) * (prod))
		dog = a * (f1-f2)
		dog_mean = np.mean(dog)
		dog = dog - dog_mean
		dog_max = np.max(dog)
		dog = dog / dog_max
		dog_tensor = torch.from_numpy(dog)
		return dog_tensor.float()